In [15]:
import numpy as np
import pandas as pd
from sklearn.utils import resample
import random

In [155]:
random.seed(666)
df = pd.read_stata('D:\\Temp\\mturk_clean_data_short.dta')
emp_moments = np.array(np.round(df.groupby("treatment").mean("buttonpresses")))
#emp_sd = np.array(np.round(df.groupby("treatment").std("buttonpresses")))

In [36]:
# Treatments are as follows:
# 1.1: benchmark specification with piece rate of 0.01 
# 1.2: benchmark specification with piece rate of 0.10 
# 1.3: benchmark specification with piece rate of 0.00 
# 3.1: social preferences (charity) with piece rate of 0.01 
# 3.2: social preferences (charity) with piece rate of 0.10 
# 10 : social preferences (gift exchange) bonus of 40 cents (independently of nr buttonpresses)
# 4.1: time discounting with extra 0.01 paid two weeks later
# 4.2: time discounting with extra 0.01 paid four weeks later

# resample is a useful command from the sklearn library that samples with replacement our data.
# We first get a smaller dataframe containing only observations for a specific treatment, we then resample the observations and
# compute the rounded mean of buttonpresses, save the result and then pass onto the next treatment. we do this for 'number' times.
def mybootstrap(dataset, number):
    
    E11, E12, E13, E31, E32, E10, E41, E42 = [], [], [], [], [], [], [], []
    box={'1.1':E11,'1.2':E12,'1.3':E13,'3.1':E31,'3.2':E32,'10':E10,'4.1':E41,'4.2':E42}
    
    for i in range(1,number+1):
        for a,b in box.items():
            db = dataset[dataset.treatment==a]
            bootsample = resample(db['buttonpresses'],replace=True,)
            b.append(np.round(np.mean(bootsample)))
    return E11, E12, E13, E31, E32, E10, E41, E42

E11, E12, E13, E31, E32, E10, E41, E42 = mybootstrap(df,2000)




In [163]:
import numpy as np

def mymindisest(params):
    # Define constants
    P = [0, 0.01, 0.1]
    expr = {
        'Exp': {
            'log_k': lambda E11, E12: (np.log(P[2]) - np.log(P[1]) * E12 / E11) / (1 - E12 / E11),
            'log_gamma': lambda log_k, E11: np.log((np.log(P[1]) - log_k) / E11),
            'log_s': lambda log_gamma, E13, log_k: np.exp(log_gamma) * E13 + log_k,
            'EG31': lambda E31, g: np.exp(E31 * g),'EG32': lambda E32, g: np.exp(E32 * g),'EG10': lambda E10, g: np.exp(E10 * g),'EG41': lambda E41, g: np.exp(E41 * g),'EG42': lambda E42, g: np.exp(E42 * g),
        


        },
        'Power': {
            'log_k': lambda E11, E12: (np.log(P[2]) - np.log(P[1]) * np.log(E12) / np.log(E11)) / (1 - np.log(E12) / np.log(E11)),
            'log_gamma': lambda log_k, E11: np.log((np.log(P[1]) - log_k) / np.log(E11)),
            'log_s': lambda log_gamma, E13, log_k: np.exp(log_gamma) * np.log(E13) + log_k,
            'EG31': lambda E31, g: E31 ** g,'EG32': lambda E32, g: E32 ** g,'EG10': lambda E10, g: E10 ** g,'EG41': lambda E41, g: E41 ** g,'EG42': lambda E42, g: E42 ** g
        }
    }

    # Extract arguments from params dictionary
    E11 = np.array(params['E11'])
    E12 = np.array(params['E12'])
    E13 = np.array(params['E13'])
    E31 = np.array(params['E31'])
    E32 = np.array(params['E32'])
    E10 = np.array(params['E10'])
    E41 = np.array(params['E41'])
    E42 = np.array(params['E42'])
    #for a,b in params.items():
        #a = params[f"{a}"]
    specification = params['specification']

    # Calculate k, gamma, and s
    log_k = expr[specification]['log_k'](E11, E12)
    log_gamma = expr[specification]['log_gamma'](log_k, E11)
    log_s = expr[specification]['log_s'](log_gamma, E13, log_k)
    k = np.exp(log_k)
    g = np.exp(log_gamma)
    s = np.exp(log_s)
    EG31 = expr[specification]['EG31'](E31,g)
    EG32 = expr[specification]['EG32'](E32,g)
    EG10 = expr[specification]['EG10'](E10,g)
    EG41 = expr[specification]['EG41'](E41,g)
    EG42 = expr[specification]['EG42'](E42,g)
    alpha = 100/9*k*(EG32-EG31)
    a = 100*k*EG31-100*s-alpha
    s_ge = k*EG10 - s
    delta = np.sqrt((k*EG42-s)/(k*EG41-s))
    beta  = 100*(k*EG41-s)/(delta**2)
    return k, g, s, alpha, a, s_ge, beta, delta



In [164]:
#E11,E12,E13,E31,E32,E10,E41,E42,specification


params = {'E11':emp_moments[0],'E12':emp_moments[1],'E13':emp_moments[2],'E31':emp_moments[6],'E32':emp_moments[7],
          "E10":emp_moments[4],'E41':emp_moments[8],'E42':emp_moments[9],'specification':'Exp'}
Table5Exp = np.array(mymindisest(params)).flatten()
params['specification'] = 'Power'
Table5Power = np.array(mymindisest(params)).flatten()
Table5Exp


array([1.26689239e-16, 1.57711308e-02, 3.31549170e-06, 3.07335179e-03,
       1.42605422e-01, 8.57890628e-06, 1.15234320e+00, 7.64690724e-01])

## 3. Estimation

### Point Estimates and Standard Errors

We now compute the minimum distance estimates for Table 5 and the standard errors via a bootstrap procedure.


In [147]:
vmindisest = np.vectorize(mymindisest)
params['specification'] = "Exp"
estimatesExp = vmindisest(params)
#mean_sd = np.zeros((8,2))
#for i in range(0,8):
    #mean_sd[i,0],mean_sd[i,1] = np.nanmean(estimatesExp[i]), np.nanstd(estimatesExp[i])

In [191]:
nw_params = {"E11":E11,"E12":E12,"E13":E13,"E31":E31,"E32":E32,"E10":E10,"E41":E41,"E42":E42,"specification":"Exp"}
vmindisest = np.vectorize(mymindisest)
estimatesExp = vmindisest(nw_params)
#np.nanstd(estimatesExp[2])

In [197]:
def cal_res(params,type):
    if type == "Exp":
        params['specification'] = "Exp"
        estimates = vmindisest(params)
    else:
        params['specification'] = "Power"
        estimates = vmindisest(params)
    res = np.zeros((8,2))
    for i in range(0,8):
        res[i,0],res[i,1] = np.mean(estimates[i][~np.isnan(estimates[i])&~np.isinf(estimates[i])]), np.std(estimates[i][~np.isnan(estimates[i])&~np.isinf(estimates[i])])
    return res


In [201]:
exp_res = cal_res(nw_params,type="Exp")
power_res = cal_res(nw_params,type="Power")
power_res

array([[5.29916296e-66, 2.36126973e-64],
       [3.61364553e+01, 1.40504784e+01],
       [4.47032630e-06, 1.06819280e-05],
       [4.05767106e-03, 1.39230123e-02],
       [1.60927839e-01, 1.49969023e-01],
       [1.11003383e-05, 2.21858031e-05],
       [2.19700324e+00, 3.74052341e+00],
       [7.98969162e-01, 3.05779577e-01]])

In [206]:
params_name = ["Level k of cost of effort", "Curvature γ of cost function","Intrinsic motivation s","Social preferences α",
                "Warm glow coefficient a","Gift exchange Δs", "Present bias β","(Weekly) discount factor δ"]
sd_exp   = exp_res[0:8,1]
sd_power = power_res[0:8,1]

In [208]:
Table5Results = pd.DataFrame({'Parameters name': params_name,
                              'Minimum dist est on average effort Power point estimates': Table5Power,
                              'Minimum dist est on average effort Power standard errors': sd_power,
                              'Minimum dist est on average effort Exp point estimates': Table5Exp,
                              'Minimum dist est on average effort Exp standard errors': sd_exp})
display(Table5Results)

,Parameters name,Minimum dist est on average effort Power point estimates,Minimum dist est on average effort Power standard errors,Minimum dist est on average effort Exp point estimates,Minimum dist est on average effort Exp standard errors
0,Level k of cost of effort,2.538986e-112,2.361270e-64,1.266892e-16,1.138363e-11
1,Curvature γ of cost function,3.313758e+01,1.405048e+01,1.577113e-02,6.649446e-03
2,Intrinsic motivation s,7.124221e-07,1.068193e-05,3.315492e-06,2.463057e-05
3,Social preferences α,2.988651e-03,1.392301e-02,3.073352e-03,1.372873e-02
4,Warm glow coefficient a,1.250433e-01,1.499690e-01,1.426054e-01,1.532168e-01
5,Gift exchange Δs,3.263487e-06,2.218580e-05,8.578906e-06,3.598378e-05
6,Present bias β,1.169033e+00,3.740523e+00,1.152343e+00,1.528536e+01
7,(Weekly) discount factor δ,7.530978e-01,3.057796e-01,7.646907e-01,2.968223e-01


In [210]:
from decimal import Decimal
columns = [Table5Power, sd_power, Table5Exp, sd_exp]
Table5Results = pd.DataFrame({'Parameters name': params_name,
                              'Minimum dist est on average effort Power point estimates': ['{0:.2e}'.format(Decimal(col[0])) for col in columns],
                              'Minimum dist est on average effort Power standard errors': np.round(columns[1], 3),
                              'Minimum dist est on average effort Exp point estimates': ['{0:.2e}'.format(Decimal(col[2])) for col in columns],
                              'Minimum dist est on average effort Exp standard errors': np.round(columns[3], 3)})
from IPython.display import display
print('Table 5: Estimates of behavioural parameters I: Mturkers actual effort. Minimum distance estimates')
display(Table5Results)

ValueError: All arrays must be of the same length

In [209]:
# Print the results

# Formatting the results nicely for the table

from decimal import Decimal

columns = [Table5Power, sd_power, Table5Exp, sd_exp]
vs = []
for col in columns:
    col = ['{0:.2e}'.format(Decimal(col[0])), round(col[1],3), '{0:.2e}'.format(Decimal(col[2])),
           round(col[3],3), round(col[4],3), '{0:.2e}'.format(Decimal(col[5])), round(col[6],2), round(col[7],2)]
    vs.append(col)
    
Table5Results = pd.DataFrame({'Parameters name': params_name,
                              'Minimum dist est on average effort Power point estimates': vs[0],
                              'Minimum dist est on average effort Power standard errors': vs[1],
                              'Minimum dist est on average effort Exp point estimates': vs[2],
                              'Minimum dist est on average effort Exp standard errors': vs[3]})
    
# Standard errors are different since the seed we used for the bootstrap procedure is different from the one used by the authors since 
# random generation across softwares/languages is not easily replicated (each software uses its own algorithm)

from IPython.display import display
print('Table 5: Estimates of behavioural parameters I: Mturkers actual effort. Minimum distance estimates')
display(Table5Results)

Table 5: Estimates of behavioural parameters I: Mturkers actual effort. Minimum distance estimates


,Parameters name,Minimum dist est on average effort Power point estimates,Minimum dist est on average effort Power standard errors,Minimum dist est on average effort Exp point estimates,Minimum dist est on average effort Exp standard errors
0,Level k of cost of effort,2.54e-112,2.36e-64,1.27e-16,1.14e-11
1,Curvature γ of cost function,33.138,14.05,0.016,0.007
2,Intrinsic motivation s,7.12e-7,1.07e-5,3.32e-6,2.46e-5
3,Social preferences α,0.003,0.014,0.003,0.014
4,Warm glow coefficient a,0.125,0.15,0.143,0.153
5,Gift exchange Δs,3.26e-6,2.22e-5,8.58e-6,3.60e-5
6,Present bias β,1.17,3.74,1.15,15.29
7,(Weekly) discount factor δ,0.75,0.31,0.76,0.3


In [196]:
nw_params['specification'] = "Power"
estimatesPower = vmindisest(nw_params)
exp_result = np.zeros((8,2))
for i in range(0,8):
    exp_result[i,0],exp_result[i,1] = np.mean(estimatesExp[i][~np.isnan(estimatesExp[i])&~np.isinf(estimatesExp[i])]), np.std(estimatesExp[i][~np.isnan(estimatesExp[i])&~np.isinf(estimatesExp[i])])
exp_result

In [162]:
a = np.array(nw_params['E10'])
b = np.array(nw_params['E11'])
al = 100/9*1.5*(a-b)
al

array([-6750.        , -7116.66666667, -6583.33333333, ...,
       -5683.33333333, -6666.66666667, -7716.66666667])

In [121]:
a_list = []
re = ["k", "g", "s", "alpha", "a", "s_ge", "beta", "delta"]
for i in range(0,8):
    a_list.append(i)
for i,j in zip(a_list,re):
    a,b = f'{j}'+"_exp"+"_meann", f'{j}'+"_exp"+"_sdd"
    a,b= np.nanmean(estimatesExp[i]), np.nanstd(estimatesExp[i])


In [125]:
estimatesExp[1]

array([0.01577113])

In [141]:
import warnings
warnings.filterwarnings('ignore') # This is to avoid showing RuntimeWarning in the notebook regarding overflow. For a couple of cases in our 1000 new samples
                                  # we cannot find the results because of overflow. Losing 2-3 observations out of thousands should not change the overall mean
                                  # for the parameters
vvmindisest = np.vectorize(mindisest)
estimatesExp = vvmindisest(E11,E12,E13,E31,E32,E10,E41,E42,'Exp')
k_exp_mean, k_exp_sd = np.nanmean(estimatesExp[0]), np.nanstd(estimatesExp[0])
g_exp_mean, g_exp_sd = np.nanmean(estimatesExp[1]), np.nanstd(estimatesExp[1])
s_exp_mean, s_exp_sd = np.nanmean(estimatesExp[2]), np.nanstd(estimatesExp[2])
alpha_exp_mean, alpha_exp_sd = np.nanmean(estimatesExp[3]), np.nanstd(estimatesExp[3])
a_exp_mean, a_exp_sd = np.nanmean(estimatesExp[4]), np.nanstd(estimatesExp[4])
s_ge_exp_mean, s_ge_exp_sd = np.nanmean(estimatesExp[5]), np.nanstd(estimatesExp[5])
beta_exp_mean, beta_exp_sd = np.nanmean(estimatesExp[6]), np.nanstd(estimatesExp[6])
delta_exp_mean, delta_exp_sd = np.nanmean(estimatesExp[7]), np.nanstd(estimatesExp[7])
k_exp_sd

nan